In [ ]:
from jetbot import Robot, Camera, bgr8_to_jpeg
import cv2
import numpy as np
from IPython.display import display, Image, clear_output
import time
import dt_apriltags
from scipy.signal import medfilt
import ipywidgets as widgets

robot = Robot()
camera = Camera.instance()
detector = dt_apriltags.Detector()

## AprilTag Detection

In [ ]:
cam_mtx = [94.861, 119.511, 119.506, 114.969]
AT_size = .065

In [ ]:
while True:
    clear_output(wait = True)
    image = np.array(camera.value)
    gray_frame = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    detections = detector.detect(gray_frame, True, cam_mtx, AT_size)

    for detection in detections:
        corners = detection.corners
        corners = [(int(x), int(y)) for (x, y) in corners]

        cv2.polylines(image, [np.array(corners)], isClosed=True, color=(0, 255, 0), thickness=2)

        tag_id = detection.tag_id
        cv2.putText(image, f"ID: {tag_id}", (int(corners[0][0]), int(corners[0][1])-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    
    display(Image(data=bgr8_to_jpeg(image)))
    if detections:
        display(f"ID: {detections[0].tag_id}")
        display(f"Translation Pose: {detections[0].pose_t}")
        display(f"Rotation Pose: {detections[0].pose_R}")
    time.sleep(0.05)


## Stop Sign Detection

In [ ]:
while True:

    clear_output(wait = True)
    orig_image = np.array(camera.value)
    image = orig_image

    # Convert to HSV (Hue, Saturation, Value) color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define range of red color in HSV space
    lower_red1 = np.array([0, 120, 70])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 120, 70])
    upper_red2 = np.array([180, 255, 255])

    # Create a mask for red color
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask = mask1 | mask2

    # Apply the mask to the image
    red_traffic_signs = cv2.bitwise_and(image, image, mask=mask)

    # Dilate image to get rid of white stop
    dilated_image = cv2.dilate(red_traffic_signs, np.ones((5, 5), np.uint8), iterations = 2)
    eroded_image = cv2.erode(dilated_image, np.ones((5, 5), np.uint8))

    # # Define the sharpening kernel
    # kernel = np.array([[-1, -1, -1],
    #                 [-1, 9, -1],
    #                 [-1, -1, -1]])

    # # Apply the kernel to the image using filter2D
    # sharpened_image = cv2.filter2D(eroded_image, -1, kernel)

    # Convert to grayscale
    gray = cv2.cvtColor(dilated_image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Detect edges using Canny edge detector
    edges = cv2.Canny(blurred, 100, 200)

    # Find contours
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Minimum area we look at
    min_area = 800

    # Loop through contours
    for contour in contours:
        contour_area = cv2.contourArea(contour)
        if contour_area > min_area:
            # Approximate the contour to a polygon
            epsilon = 0.01 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)

            # If the shape has 8 vertices, it could be a stop sign (octagon)
            if len(approx) == 8:
                cv2.drawContours(orig_image, [approx], -1, (0, 255, 0), 2)
                cv2.putText(orig_image, 'Stop Sign', tuple(approx[0][0]), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                cv2.putText(orig_image, str(contour_area), (approx[0][0][0], approx[0][0][1] + 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    
    display(Image(data=bgr8_to_jpeg(orig_image)))
    time.sleep(0.05)
    

## Lane Detection

In [ ]:
def adjust_gamma(image, gamma=1.2):
    invGamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** invGamma * 255 for i in np.arange(256)]).astype("uint8")
    return cv2.LUT(image, table)

def iqr_filter(x_list):
    x = np.array(x_list)
    q1, q3 = np.percentile(x, [25, 75])
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return [val for val in x if lower <= val <= upper]

def get_lane_midline_inner_edges_outlier_debug(image, debug_prefix='debug', apply_gamma=True):
    h, w = image.shape[:2]
    mid_x = w // 2

    if apply_gamma:
        image = adjust_gamma(image, gamma=1.2)

    # === Sobel X edges ===
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobelx = np.absolute(sobelx)
    sobelx = np.uint8(255 * sobelx / np.max(sobelx))
    _, sobel_bin = cv2.threshold(sobelx, 40, 255, cv2.THRESH_BINARY)

    sobel_mask = np.zeros_like(sobel_bin)
    sobel_mask[int(h * 0.45):, :] = sobel_bin[int(h * 0.45):, :]
    sobel_mask = cv2.morphologyEx(sobel_mask, cv2.MORPH_CLOSE, np.ones((3, 3), np.uint8), iterations=1)

    # === HSV yellow mask ===
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv[..., 2] = cv2.equalizeHist(hsv[..., 2])
    mask_yellow = cv2.inRange(hsv, (10, 30, 40), (45, 255, 255))
    mask_yellow = cv2.morphologyEx(mask_yellow, cv2.MORPH_CLOSE, np.ones((3, 3), np.uint8), iterations=1)
    yellow_sobel = cv2.bitwise_and(mask_yellow, sobel_mask)

    # === Extract left (yellow ∩ sobel) and right (sobel only) points ===
    left_pts = []
    right_pts = []

    contours, _ = cv2.findContours(sobel_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        if cv2.contourArea(cnt) < 20:
            continue
        for x, y in cnt[:, 0, :]:
            if y < int(h * 0.45):
                continue
            if x < mid_x and yellow_sobel[y, x] > 0:
                left_pts.append((x, y))
            elif x > mid_x:
                right_pts.append((x, y))

    # === Overlay image ===
    out_img = image.copy()
    if out_img.ndim == 2:
        out_img = cv2.cvtColor(out_img, cv2.COLOR_GRAY2BGR)

    mid_pts = []
    max_slope_deg = 30
    prev_xc, prev_y = None, None

    for y in range(int(h * 0.45), h, 4):
        x_lefts = [x for (x, yy) in left_pts if abs(yy - y) <= 2]
        x_rights = [x for (x, yy) in right_pts if abs(yy - y) <= 2]

        if len(x_lefts) >= 3:
            x_lefts = iqr_filter(x_lefts)
        if len(x_rights) >= 3:
            x_rights = iqr_filter(x_rights)

        if not x_lefts or not x_rights:
            continue

        x_left = max(x_lefts)
        x_right = min(x_rights)
        x_center = (x_left + x_right) // 2

        # Slope filtering
        if prev_xc is not None:
            dx = x_center - prev_xc
            dy = y - prev_y
            angle = np.degrees(np.arctan2(abs(dx), dy + 1e-5))
            if angle > max_slope_deg:
                continue

        mid_pts.append((x_center, y))
        prev_xc, prev_y = x_center, y

        # Draw edges
        cv2.circle(out_img, (int(x_left), int(y)), 2, (255, 0, 0), -1)
        cv2.circle(out_img, (int(x_right), int(y)), 2, (0, 0, 255), -1)

    # === Midline smoothing ===
    if mid_pts:
        x_vals = [x for x, y in mid_pts]
        x_smooth = medfilt(x_vals, kernel_size=5)
        mid_pts = list(zip(x_smooth, [y for _, y in mid_pts]))

    for x_c, y in mid_pts:
        cv2.circle(out_img, (int(x_c), int(y)), 2, (0, 255, 0), -1)

    # === Save debug outputs ===
    if debug_prefix:
        cv2.imwrite(f"{debug_prefix}_sobel.jpg", sobel_bin)
        cv2.imwrite(f"{debug_prefix}_yellow.jpg", mask_yellow)
        cv2.imwrite(f"{debug_prefix}_final_combined.jpg", sobel_mask)
        cv2.imwrite(f"{debug_prefix}_overlay.jpg", out_img)

    # === Deviation computation ===
    if mid_pts:
        x_center_last = mid_pts[-1][0]
        deviation = (x_center_last - mid_x) / mid_x
    else:
        deviation = 0.0

    return out_img, deviation


In [ ]:
import threading
image = widgets.Image(format='jpeg', width=300, height=300)
display(image)

def update_image():
    while True:
        img = camera.value
        out_img, dev = get_lane_midline_inner_edges_outlier_debug(img, None)
        image.value = bgr8_to_jpeg(out_img)
        display(dev)
        time.sleep(0.01)

threading.Thread(target=update_image, daemon=True).start()